In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# Step 2: Load data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
train_data
test_data

In [ ]:
train_data.info()

In [ ]:
# we drop the columns that are object and have no or little effect on the feture we want to predict

train_data = train_data.drop(columns=['upload_date','has_subtitles', 'is_ads_enabled', 'is_comments_enabled',
                                      'is_age_limit', 'is_live_content', 'description', 'title']).reset_index(drop=True)
test_data = test_data.drop(columns=['upload_date','has_subtitles', 'is_ads_enabled', 'is_comments_enabled',
                                      'is_age_limit', 'is_live_content', 'description', 'title']).reset_index(drop=True)


In [ ]:
train_data.describe()

In [ ]:
train_data =train_data[train_data['uploader_sub_count']>0]
test_data =test_data[test_data['uploader_sub_count']>0]

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
# We want to see the correlation between the numerical columns and the dislike column. 
import matplotlib.pyplot as plt
import seaborn as sns

# First plot
plt.figure()
sns.scatterplot(x=train_data['uploader_sub_count'],
                y=train_data['dislike_count'], alpha=0.3)
plt.title('Uploader Sub Count vs Dislike Count')

# Second plot
plt.figure()
sns.scatterplot(x=train_data['view_count'],
                y=train_data['dislike_count'], alpha=0.5)
plt.title('View Count vs Dislike Count')

# Third plot
plt.figure()
sns.scatterplot(x=train_data['like_count'],
                y=train_data['dislike_count'], alpha=0.5)
plt.title('Like Count vs Dislike Count')

# Show the plots
plt.show()


In [ ]:

# Sample data
# Assuming 'train_data' is your DataFrame
# Create a correlation matrix
corr_matrix = train_data.corr()

# Plot heatmap
plt.figure(figsize=(8, 5))
plt.imshow(corr_matrix, cmap='coolwarm')
plt.colorbar()  # Add colorbar to show the scale

plt.show()
 # to see the correlation between features we show it in table format. 
print('The corrolation between the uploader_sub_count and the columns is small. in the scatterplot above, we can see this as well. ')
corr_matrix

In [ ]:

# Assuming 'train_data' is your DataFrame and 'X' is the feature you want to transform
# Apply log transformation
train_data['log_uploader_sub_count'] = np.log1p(train_data['uploader_sub_count'])
train_data['log_uploader_sub_count'].hist()


In [ ]:
from sklearn.preprocessing import StandardScaler

# Assuming 'train_data' is your DataFrame and 'X' is the feature you want to standardize
scaler = StandardScaler()
train_data['zscore_view_count'] = scaler.fit_transform(train_data[['view_count']])

train_data['zscore_view_count'].hist()


In [ ]:
plt.figure()
sns.scatterplot(x=train_data['log_uploader_sub_count'],
                y=train_data['dislike_count'], alpha=0.7)
plt.title('log_uploader_sub_count vs Dislike Count')
plt.plot([0, 20], [0, 175000], '--', color='red', linewidth=2)


In [ ]:
plt.figure()
sns.scatterplot(x=train_data['zscore_view_count'],
                y=train_data['dislike_count'], alpha=0.7)
plt.title('View Count vs Dislike Count')
plt.plot([0, 500], [0, 600000], '--', color='red', linewidth=2)


In [ ]:
import pandas as pd

# Calculate the 1st and 3rd quartiles (Q1 and Q3)
Q1 = train_data['view_count'].quantile(0.25)
Q3 = train_data['view_count'].quantile(0.75)
IQR = Q3 - Q1

# Apply IQR scaling
train_data['iqr_scaled_view_count'] = (train_data['view_count'] - Q1) / IQR

# Check the distribution after scaling
train_data['iqr_scaled_view_count'].hist()


In [ ]:
plt.figure()
sns.scatterplot(x=train_data['iqr_scaled_view_count'],
                y=train_data['dislike_count'], alpha=0.7)
plt.title('iqr_scaled_view_count vs Dislike Count')
plt.plot([0, 400000], [0, 600000], '--', color='red', linewidth=2)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

# Assuming train_data is your original dataset

# Step 1: Log Transformation since the data is scattered like exponential function
train_data['log_uploader_sub_count'] = np.log(train_data['uploader_sub_count'] + 1)

# Step 2: Z-Score Normalization
scaler = StandardScaler()
train_data['zscore_view_count'] = scaler.fit_transform(train_data[['view_count']])

# Step 3: IQR Scaling
Q1 = train_data['view_count'].quantile(0.25)
Q3 = train_data['view_count'].quantile(0.75)
IQR = Q3 - Q1
train_data['iqr_scaled_view_count'] = (train_data['view_count'] - Q1) / IQR

# Step 4: Define X and y for regression
X = train_data[['log_uploader_sub_count', 'zscore_view_count', 'iqr_scaled_view_count']]
y = train_data['dislike_count']

# Step 5: Split the data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to calculate metrics and return coefficients and intercepts
def evaluate_model(model, X, y_true):
    y_pred = model.predict(X)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # Return coefficients and intercept for linear models
    if hasattr(model, 'coef_'):
        coefficients = model.coef_
        intercept = model.intercept_
    else:
        coefficients = None
        intercept = None
    
    return mae, rmse, r2, coefficients, intercept

# Models
models = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression (Degree 2)': LinearRegression(),
    'Ridge Regression': Ridge(),
}

# Fit models and store results
results = {}

# Linear Regression
linear_reg = models['Linear Regression'].fit(X_train_split, y_train_split)
results['Linear Regression'] = evaluate_model(linear_reg, X_val_split, y_val_split)

# Polynomial Regression (Degree 2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_split)
X_val_poly = poly.transform(X_val_split)
poly_reg = models['Polynomial Regression (Degree 2)'].fit(X_train_poly, y_train_split)
results['Polynomial Regression (Degree 2)'] = evaluate_model(poly_reg, X_val_poly, y_val_split)

# Ridge Regression
ridge_reg = models['Ridge Regression'].fit(X_train_split, y_train_split)
results['Ridge Regression'] = evaluate_model(ridge_reg, X_val_split, y_val_split)

# Display the evaluation metrics along with coefficients and intercepts
for model_name, (mae, rmse, r2, coefficients, intercept) in results.items():
    print(f"\n{model_name} Results:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}")
    if coefficients is not None:
        print(f"Coefficients: {coefficients}")
        print(f"Intercept: {intercept}")

# Visualization of the Predictions
plt.figure(figsize=(10, 6))

# Predict on validation set for visualization
y_pred_linear = linear_reg.predict(X_val_split)
y_pred_poly = poly_reg.predict(X_val_poly)
y_pred_ridge = ridge_reg.predict(X_val_split)



Based on the results in the image:

### Evaluation Metrics:
- **Mean Absolute Error (MAE)**: A lower value is better. It represents the average magnitude of the errors in the predictions.
- **Root Mean Squared Error (RMSE)**: A lower value is better. It is more sensitive to large errors than MAE.
- **R-squared (R²)**: A higher value is better, as it shows how well the model explains the variance in the target variable (with 1.0 being a perfect fit).

### Model Comparison:

#### **Linear Regression:**
- **MAE**: 18.39
- **RMSE**: 455.23
- **R²**: 0.37

#### **Polynomial Regression (Degree 2):**
- **MAE**: 13.46
- **RMSE**: 455.81
- **R²**: 0.31

#### **Ridge Regression:**
- **MAE**: 18.39
- **RMSE**: 455.23
- **R²**: 0.37

### **Best Model:**

- The **Polynomial Regression (Degree 2)** model has the **lowest MAE** (13.46) but its **R²** value is lower than both Linear and Ridge Regression.
- The **Linear Regression** and **Ridge Regression** models have identical performance in terms of RMSE (455.23) and R² (0.37).

### Conclusion:
- If you are primarily concerned with **minimizing MAE** (errors), **Polynomial Regression** is the best.
- If you want a **better fit** in terms of **explained variance** (R²), **Linear Regression** or **Ridge Regression** would be better since they have higher R².

Since the RMSE values are almost identical across the models, **Polynomial Regression** could be chosen for better error minimization (lower MAE).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

# Assuming train_data is your original dataset

# Step 1: Log Transformation
train_data['log_uploader_sub_count'] = np.log(train_data['uploader_sub_count'] + 1)

# Step 2: Z-Score Normalization
scaler = StandardScaler()
train_data['zscore_view_count'] = scaler.fit_transform(train_data[['view_count']])

# Step 3: IQR Scaling
Q1 = train_data['view_count'].quantile(0.25)
Q3 = train_data['view_count'].quantile(0.75)
IQR = Q3 - Q1
train_data['iqr_scaled_view_count'] = (train_data['view_count'] - Q1) / IQR

# Step 4: Define X and y for regression
X = train_data[['log_uploader_sub_count', 'zscore_view_count', 'iqr_scaled_view_count']]
y = train_data['dislike_count']

# Step 5: Split the data into training, validation, and test sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_split, y_train_split, test_size=0.25, random_state=42)

# Function to calculate metrics and return coefficients and intercepts
def evaluate_model(model, X, y_true, round_predictions=False):
    y_pred = model.predict(X)

    # Round the predicted values to integers since the number of dislikes is an integer 
    if round_predictions:
        y_pred = np.round(y_pred).astype(int)
        # Ensure no negative predictions (dislike counts must be non-negative)
        y_pred = np.maximum(y_pred, 0)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # Return coefficients, intercept, and predictions for linear models
    if hasattr(model, 'coef_'):
        coefficients = model.coef_
        intercept = model.intercept_
    else:
        coefficients = None
        intercept = None
    
    return mae, rmse, r2, coefficients, intercept, y_pred  # Include y_pred in the return

# Models
models = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression (Degree 2)': LinearRegression(),
    'Ridge Regression': Ridge(),
}

# Fit models and store results
results = {}

# Linear Regression
linear_reg = models['Linear Regression'].fit(X_train_split, y_train_split)
results['Linear Regression'] = evaluate_model(linear_reg, X_val_split, y_val_split, round_predictions=True)

# Polynomial Regression (Degree 2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_split)
X_val_poly = poly.transform(X_val_split)
poly_reg = models['Polynomial Regression (Degree 2)'].fit(X_train_poly, y_train_split)
results['Polynomial Regression (Degree 2)'] = evaluate_model(poly_reg, X_val_poly, y_val_split, round_predictions=True)

# Ridge Regression
ridge_reg = models['Ridge Regression'].fit(X_train_split, y_train_split)
results['Ridge Regression'] = evaluate_model(ridge_reg, X_val_split, y_val_split, round_predictions=True)

# Display the evaluation metrics along with coefficients and intercepts for validation data
for model_name, (mae, rmse, r2, coefficients, intercept, y_pred_val) in results.items():
    print(f"\n{model_name} Results on Validation Data:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}")
    if coefficients is not None:
        print(f"Coefficients: {coefficients}")
        print(f"Intercept: {intercept}")
    print(f"Predicted Values (Validation): {y_pred_val[:10]}")  # Print first 10 predictions

# Now apply models to the test data
test_results = {}

# Linear Regression on test data
test_results['Linear Regression'] = evaluate_model(linear_reg, X_test_split, y_test_split, round_predictions=True)

# Polynomial Regression on test data
X_test_poly = poly.transform(X_test_split)
test_results['Polynomial Regression (Degree 2)'] = evaluate_model(poly_reg, X_test_poly, y_test_split, round_predictions=True)

# Ridge Regression on test data
test_results['Ridge Regression'] = evaluate_model(ridge_reg, X_test_split, y_test_split, round_predictions=True)

# Display the evaluation metrics along with coefficients and intercepts for test data
for model_name, (mae, rmse, r2, coefficients, intercept, y_pred_test) in test_results.items():
    print(f"\n{model_name} Results on Test Data:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}")
    if coefficients is not None:
        print(f"Coefficients: {coefficients}")
        print(f"Intercept: {intercept}")
    print(f"Predicted Values (Test): {y_pred_test[:10]}")  # Print first 10 predictions


### Interpretation:
1. **Linear Regression:** Performs similarly in both validation and test sets, with an R² of 0.48 on validation and 0.20 on test data. This suggests the model captures some variance but may not generalize well.
   
2. **Polynomial Regression (Degree 2):** Has better performance (lower MAE) on both validation and test data compared to linear regression. However, the R² suggests that while it fits the data better (especially the validation data), it might overfit and not generalize well to the test data.

3. **Ridge Regression:** Results are identical to linear regression, which suggests the regularization strength might be too low or the data doesn't benefit significantly from regularization.

The relatively low R² values on test data (especially for linear and ridge regressions) indicate that the models are not explaining a large portion of the variance in the data, which might suggest either noise in the data or that the models might be too simple for the problem at hand.

# The Code below is the same code without rounding option which gives the predicted values an a real number 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt

# Assuming train_data is your original dataset

# Step 1: Log Transformation
train_data['log_uploader_sub_count'] = np.log(train_data['uploader_sub_count'] + 1)

# Step 2: Z-Score Normalization
scaler = StandardScaler()
train_data['zscore_view_count'] = scaler.fit_transform(train_data[['view_count']])

# Step 3: IQR Scaling
Q1 = train_data['view_count'].quantile(0.25)
Q3 = train_data['view_count'].quantile(0.75)
IQR = Q3 - Q1
train_data['iqr_scaled_view_count'] = (train_data['view_count'] - Q1) / IQR

# Step 4: Define X and y for regression
X = train_data[['log_uploader_sub_count', 'zscore_view_count', 'iqr_scaled_view_count']]
y = train_data['dislike_count']

# Step 5: Split the data into training, validation, and test sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_split, y_train_split, test_size=0.25, random_state=42)

# Function to calculate metrics and return coefficients, intercept, and predictions
def evaluate_model(model, X, y_true):
    y_pred = model.predict(X)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    # Return coefficients, intercept, and predictions for linear models
    if hasattr(model, 'coef_'):
        coefficients = model.coef_
        intercept = model.intercept_
    else:
        coefficients = None
        intercept = None
    
    return mae, rmse, r2, coefficients, intercept, y_pred  # Include y_pred in the return

# Models
models = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression (Degree 2)': LinearRegression(),
    'Ridge Regression': Ridge(),
}

# Fit models and store results
results = {}

# Linear Regression
linear_reg = models['Linear Regression'].fit(X_train_split, y_train_split)
results['Linear Regression'] = evaluate_model(linear_reg, X_val_split, y_val_split)

# Polynomial Regression (Degree 2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_split)
X_val_poly = poly.transform(X_val_split)
poly_reg = models['Polynomial Regression (Degree 2)'].fit(X_train_poly, y_train_split)
results['Polynomial Regression (Degree 2)'] = evaluate_model(poly_reg, X_val_poly, y_val_split)

# Ridge Regression
ridge_reg = models['Ridge Regression'].fit(X_train_split, y_train_split)
results['Ridge Regression'] = evaluate_model(ridge_reg, X_val_split, y_val_split)

# Display the evaluation metrics along with coefficients, intercepts, and predictions for validation data
for model_name, (mae, rmse, r2, coefficients, intercept, y_pred_val) in results.items():
    print(f"\n{model_name} Results on Validation Data:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}")
    if coefficients is not None:
        print(f"Coefficients: {coefficients}")
        print(f"Intercept: {intercept}")
    print(f"Predicted Values (Validation): {y_pred_val[:10]}")  # Print first 10 predictions

# Now apply models to the test data
test_results = {}

# Linear Regression on test data
test_results['Linear Regression'] = evaluate_model(linear_reg, X_test_split, y_test_split)

# Polynomial Regression on test data
X_test_poly = poly.transform(X_test_split)
test_results['Polynomial Regression (Degree 2)'] = evaluate_model(poly_reg, X_test_poly, y_test_split)

# Ridge Regression on test data
test_results['Ridge Regression'] = evaluate_model(ridge_reg, X_test_split, y_test_split)

# Display the evaluation metrics along with coefficients, intercepts, and predictions for test data
for model_name, (mae, rmse, r2, coefficients, intercept, y_pred_test) in test_results.items():
    print(f"\n{model_name} Results on Test Data:")
    print(f"MAE: {mae}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}")
    if coefficients is not None:
        print(f"Coefficients: {coefficients}")
        print(f"Intercept: {intercept}")
    print(f"Predicted Values (Test): {y_pred_test[:10]}")  # Print first 10 prediction
